<html><h2><center>Le code d'extraction des tweets utilise Selenium pour automatiser l'interaction avec Twitter et récupérer les tweets en temps réel</center><h2></html>

<html>Pour l'automatisation du navigateur web, il utilise <span style="color:blue">Selenium</span>. Les modules importés de <span style="color:blue">Selenium</span> incluent le module principal <span style="color:blue">Selenium</span>, la classe <span style="color:red">webdriver</span> pour contrôler le navigateur, les éléments de localisation By pour trouver les éléments sur une page web, les touches Keys pour simuler des actions clavier, et les modules Options spécifiques au navigateur pour personnaliser le comportement de celui-ci. Le module time est également importé pour gérer les délais.<html>

In [1]:
# pip install selenium

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


<html>Pour l'interaction avec une base de données <span style="color:blue">MongoDB</span>, il utilise <span style="color:red">pymongo</span>. Les modules importés incluent <span style="color:red">pymongo</span> pour la connexion et la manipulation de la base de données, et <span style="color:blue">MongoClient</span> pour établir une connexion avec le serveur <span style="color:blue">MongoDB</span>.</html>

In [2]:
import pymongo
from pymongo import MongoClient

Ouverture du navigateur et navigation vers Twitter: Il démarre un navigateur Edge à l'aide de Selenium et ouvre la page d'accueil de Twitter.

In [3]:
options = webdriver.EdgeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Edge(options=options)
driver.get("https://twitter.com/home")

Connexion à un compte Twitter: Il entre le nom d'utilisateur et le mot de passe dans les champs correspondants sur la page de connexion de Twitter, puis clique sur le bouton "Log in" pour se connecter au compte.

In [4]:
your_twitter_user=""
your_twitter_password=""
wait = WebDriverWait(driver, 10)
username = wait.until(EC.visibility_of_element_located((By.XPATH, "//input[@name='text']")))
username.send_keys(your_twitter_user)
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Next')]")))
next_button.click()

password = wait.until(EC.visibility_of_element_located((By.XPATH, "//input[@name='password']")))
password.send_keys(your_twitter_password)
log_in = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Log in')]")))
log_in.click()

Recherche de tweets en anglais: Il recherche des tweets en langue anglaise en utilisant une requête vide, suivie de " lang:en", pour filtrer les tweets en anglais.

In [41]:
subject = ""
search_query = subject + " lang:en"
search_box = wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@data-testid="SearchBox_Search_Input"]')))
search_box.send_keys(search_query)
search_box.send_keys(Keys.ENTER)

Sélection de l'onglet "Latest": Il clique sur l'onglet "Latest" pour afficher les tweets les plus récents correspondant à la requête de recherche.

In [42]:
latest_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Latest')]")))
latest_button.click()

<html>Nous établissons une connexion à MongoDB sur l'hôte local (localhost) et le port 27017, puis nous sélectionnons la base de données <span style="color:blue;">'tweets_db'</span> à partir de cette connexion, et enfin, nous sélectionnons la collection <span style="color:blue;">'tweets'</span> dans la base de données 'tweets_db'
</html>

In [46]:
client = MongoClient('localhost', 27017)
db = client['tweets_db']
collection = db['tweets']
# collection.delete_many({})
# 

<html>
Ici, nous extrayons des données à partir de tweets sur <span style="color:blue;">Twitter</span> en utilisant une boucle infinie pour continuer le scraping jusqu'à ce qu'un certain nombre de tweets soit atteint. À chaque itération de la boucle, nous recherchons les éléments HTML correspondant aux <span style="color:red;">tweets</span>, aux <span style="color:red;">horodatages</span>, aux nombres de <span style="color:red;">likes</span>, aux nombres de <span style="color:red;">retweets</span> à l'aide de XPath. Ensuite, nous insérons ces données dans une collection <span style="color:blue;">MongoDB</span>.
</html>

In [49]:
tweet_count = 0
while True:
    tweets = driver.find_elements(By.XPATH, '//div[@data-testid="tweetText"]')
    tweets_time = driver.find_elements(By.XPATH, '//time[@datetime]')
    likes = driver.find_elements(By.XPATH, './/div[@data-testid="like"]')
    retweets = driver.find_elements(By.XPATH, './/div[@data-testid="retweet"]')

    for tweet, tweet_time,like,ret in zip(tweets, tweets_time,likes,retweets):
        # print(lik.text,ret.text)
        if not like or like.text == "":
            nbLike=0
        else:
            nbLike=like.text
        if not ret or ret.text == "":
            nbRet=0
        else:
            nbRet=ret.text

        t = {
            "content": tweet.text,
            "date": tweet_time.get_attribute("datetime"),
            'likes':nbLike,
            "ret":nbRet,
        }
        # print(t)
        collection.insert_one(t)
        
        tweet_count += 1
    
    if tweet_count >=70:
            break
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    

{'content': "I'm happy for you guys", 'date': '2024-05-05T16:32:50.000Z', 'likes': '3', 'ret': 0}
{'content': 'IM NOT ASHAMED', 'date': '2024-05-05T16:32:50.000Z', 'likes': '2', 'ret': 0}
{'content': 'W aslam', 'date': '2024-05-05T16:32:50.000Z', 'likes': 0, 'ret': 0}
{'content': 'respect. ', 'date': '2024-05-05T16:32:50.000Z', 'likes': 0, 'ret': 0}
{'content': 'Happy Fathers Day', 'date': '2024-05-05T16:32:50.000Z', 'likes': 0, 'ret': 0}
{'content': 'No one cares what jih@dis says', 'date': '2024-05-05T16:32:50.000Z', 'likes': 0, 'ret': 0}
{'content': 'Keren banget fotonya! Me wants to be your moots. :]', 'date': '2024-05-05T16:32:50.000Z', 'likes': 0, 'ret': 0}
{'content': 'Check out my #SMRTG analysis on \n@TradingView\n:', 'date': '2024-05-05T16:32:50.000Z', 'likes': 0, 'ret': 0}
{'content': 'Clock it', 'date': '2024-05-05T16:32:50.000Z', 'likes': 0, 'ret': 0}
{'content': 'one like and ill jump', 'date': '2024-05-05T16:32:50.000Z', 'likes': '1', 'ret': 0}
{'content': 'at the end of

Nous récupérons tous les documents de la collection MongoDB, les affichons, puis fermons la connexion cliente.

In [50]:
docs = collection.find()
for doc in docs:
    print(doc)
# client.close()


{'_id': ObjectId('6637b4ba28e662f0e0d3db56'), 'content': 'Who are we playing next?', 'date': '2024-05-05T16:32:50.000Z', 'likes': 0, 'ret': 0}
{'_id': ObjectId('6637b4ba28e662f0e0d3db57'), 'content': 'The Rock', 'date': '2024-05-05T16:32:50.000Z', 'likes': 0, 'ret': 0}
{'_id': ObjectId('6637b4ba28e662f0e0d3db58'), 'content': 'GoodAfternoon', 'date': '2024-05-05T16:32:50.000Z', 'likes': 0, 'ret': 0}
{'_id': ObjectId('6637b4ba28e662f0e0d3db59'), 'content': 'kermit is the better dresser \n@JimHensonMuppet', 'date': '2024-05-05T16:32:50.000Z', 'likes': 0, 'ret': 0}
{'_id': ObjectId('6637b4ba28e662f0e0d3db5a'), 'content': '“The Pope shall be for the west, and you shall be for us, you shall be for us!”', 'date': '2024-05-05T16:32:50.000Z', 'likes': 0, 'ret': 0}
{'_id': ObjectId('6637b4bb28e662f0e0d3db5b'), 'content': 'W.T.A.F \n\nAre these guys climbing mountains in training?? ', 'date': '2024-05-05T16:32:50.000Z', 'likes': 0, 'ret': 0}
{'_id': ObjectId('6637b4bb28e662f0e0d3db5c'), 'content'